# Operaciones que realiza un mecanismo de atención

Primero se importan las librerías en el cabecero

In [47]:
import numpy as np
import scipy.special as sci
import pandas as pd

El corpus en este caso es inicializado de forma aleatoria y representa a los embeddings ya con codificación posicional.
En la práctica se pueden tomar estos embeddings de una librería como word2vec.

In [48]:
np.random.seed(42)  # Valor aleatorio de seed, el cual permite poder replicar los resultados
corpus = np.around(np.random.rand(3,3),3)
corpus.shape
print(f"Corpus shape: {corpus.shape}\n\n")
print(f"Corpus:\n{corpus}")

Corpus shape: (3, 3)


Corpus:
[[0.375 0.951 0.732]
 [0.599 0.156 0.156]
 [0.058 0.866 0.601]]


A partir de los embeddings iniciales se obtienen 3 matrices, Q, K y V, donde cada una es una copia de dichos embeddings.

In [49]:
Q = corpus.copy()
K = corpus.copy()
V = corpus.copy()

print(f"Q shape: {Q.shape}\n\n")
print(f"Q:\n{Q}")

Q shape: (3, 3)


Q:
[[0.375 0.951 0.732]
 [0.599 0.156 0.156]
 [0.058 0.866 0.601]]


En este caso la matriz de pesos inicia de forma aleatoria, en la práctica estas matrices pueden ya estar dadas gracias a entrenamientos previos para ahorrar tiempo y entrenamiento o bien inicializarse de forma aleatoria.

In [50]:
W_Q = np.around(np.random.rand(3,3),3)
W_K = np.around(np.random.rand(3,3),3)
W_V = np.around(np.random.rand(3,3),3)

print(f"Key weights: {W_K}\n\n")
print(f"Query weights: {W_Q}\n\n")
print(f"Values weights: {W_V}\n\n")

Key weights: [[0.432 0.291 0.612]
 [0.139 0.292 0.366]
 [0.456 0.785 0.2  ]]


Query weights: [[0.708 0.021 0.97 ]
 [0.832 0.212 0.182]
 [0.183 0.304 0.525]]


Values weights: [[0.514 0.592 0.046]
 [0.608 0.171 0.065]
 [0.949 0.966 0.808]]




Se multiplica cada matriz (Q,K,V) con su respectiva matriz de pesos.

Dicha matriz de pesos puede ser incializada (y en este caso es así) de forma aleatoria, sin embargo, estos valores se pueden tener pre-entrenados, de tal manera que ahorramos el ajuste de estos pesos.

In [51]:
Q = np.dot(Q,W_Q)
K = np.dot(K,W_K)
V = np.dot(V,W_V)

print(f"Key (Post adding weights): \n{K}\n\n")
print(f"Query (Post adding weights): \n{Q}\n\n")
print(f"Values (Post adding weights): \n{V}\n\n")

Key (Post adding weights): 
[[0.627981 0.961437 0.723966]
 [0.351588 0.342321 0.454884]
 [0.419486 0.741535 0.472652]]


Query (Post adding weights): 
[[1.190688 0.432015 0.921132]
 [0.582432 0.093075 0.691322]
 [0.871559 0.367514 0.529397]]


Values (Post adding weights): 
[[1.465626 1.091733 0.670521]
 [0.550778 0.53198  0.163742]
 [1.126689 0.762988 0.544566]]




Se tiene que realizar la siguiente operación: 
$$\text{Attention(Q,K,V)}=\text{softmax}(\frac{QK^T}{\sqrt{d_k}})$$

$\text{res}=QK^T$

In [52]:
res = np.dot(Q,K.T)
res

array([[1.8299529 , 0.98552763, 1.25520607],
       [0.9557356 , 0.55110895, 0.64009517],
       [1.28392948, 0.67305167, 0.88835184]])

$\text{res}=\frac{\text{res}}{\sqrt{d_k}}=\frac{QK^T}{\sqrt{d_k}}$

In [53]:
res = res/np.sqrt(K.shape[0]+K.shape[1])
res

array([[0.74707514, 0.40233997, 0.51243573],
       [0.39017743, 0.22498928, 0.26131776],
       [0.52416201, 0.27477219, 0.36266812]])

$\text{res}=\text{softmax}(\text{res})=\text{softmax}(\frac{QK^T}{\sqrt{d_k}})$

In [54]:
res = sci.softmax(res,axis=0)
print(f"Sum = {np.sum(res,axis=0)}")
# e_x = np.exp(res[:1] - np.max(res))
# res = e_x / e_x.sum()
# print(f"Sum = {np.sum(res)}")

Sum = [1. 1. 1.]


# Multiplicación con los valores originales
Esto se realiza para agregar la información que ya está contextualizada con la información original.

Esto se hace con el objetivo de que la matriz original contenga ya la información contextual, es decir, los tokens (en este caso palabras) ya conocen su relación con el resto de palabras del corpus.

In [55]:
res = res*V
res

array([[0.58624373, 0.40170918, 0.25409701],
       [0.15418173, 0.16393369, 0.0482712 ],
       [0.36061946, 0.24712179, 0.17766196]])

Se aplica un redondeo con el objetivo de hacer más fácil la visualización

In [56]:
res = np.around(res,4)
res

array([[0.5862, 0.4017, 0.2541],
       [0.1542, 0.1639, 0.0483],
       [0.3606, 0.2471, 0.1777]])

Se convierte en DataFrame con el fin de poder visualizar que cada embedding es una representación vectorial de los tokens y que el mecanismo de atención tiene como fin entender la relación que tienen cada palabra con las otras del corpus.

Se puede ver que en la matriz, el valor que se encuentre en la coordenada (renglón, columna) será la representación de la relación entre tokens, en donde un valor más grande indica una relación más fuerte entre palabras.

In [57]:
res_df = pd.DataFrame(res)
res_df.columns=["Amo","el","queso"]
res_df.index=["Amo","el","queso"]
res_df

,Amo,el,queso
Amo,0.5862,0.4017,0.2541
el,0.1542,0.1639,0.0483
queso,0.3606,0.2471,0.1777
